In [ ]:
import torch as th
import torch.nn as nn
from active_critic.model_src.state_model import StateModel, StateModelArgs

In [ ]:
output.shape

In [111]:
def build_sequence(rnn_predictor:th.nn.GRU, action_model:StateModel, embedding:th.Tensor, seq_len:int):
    embeddings = embedding.unsqueeze(0)
    action = action_model.forward(inpt=embedding.transpose(0,1).reshape(embedding.shape[1], -1)).unsqueeze(0)
    actions = action.unsqueeze(0)
    for i in range(seq_len):
        _, embedding = rnn_predictor(action, embedding)
        embeddings = th.cat((embeddings, embedding.unsqueeze(0)), dim=0)
        action = action_model.forward(inpt=embedding.transpose(0,1).reshape(embedding.shape[1], -1)).unsqueeze(0)
        actions = th.cat((actions, action.unsqueeze(0)), dim=0)
    return embeddings, actions

In [112]:
batch_size = 3
embedding_size = 20
action_dim = 10
layers = 2
seq_len = 1
ac_seq_len = 4


rnn = nn.GRU(action_dim, embedding_size, layers)

sma = StateModelArgs()
sma.arch = [10,action_dim]
sma.device = 'cpu'
sma.lr = 1e-3
action_model = StateModel(args=sma)

init_embeddng = th.rand([layers, batch_size, embedding_size], requires_grad=True)

embeddings, actions = build_sequence(rnn_predictor=rnn, action_model=action_model, embedding=init_embeddng, seq_len=ac_seq_len)

In [113]:
embeddings.retain_grad()
actions.retain_grad()
((embeddings[0] - th.ones_like(embeddings[0]))**2).mean().backward(retain_graph=True)

In [116]:
embeddings.grad

tensor([[[[-0.0117, -0.0032, -0.0143, -0.0022, -0.0147, -0.0096, -0.0153,
           -0.0034, -0.0032, -0.0025, -0.0131, -0.0083, -0.0157, -0.0030,
           -0.0087, -0.0098, -0.0026, -0.0120, -0.0053, -0.0043],
          [-0.0064, -0.0153, -0.0124, -0.0050, -0.0046, -0.0157, -0.0164,
           -0.0060, -0.0122, -0.0079, -0.0062, -0.0139, -0.0017, -0.0060,
           -0.0127, -0.0052, -0.0095, -0.0116, -0.0150, -0.0111],
          [-0.0141, -0.0070, -0.0156, -0.0046, -0.0102, -0.0010, -0.0116,
           -0.0164, -0.0139, -0.0090, -0.0013, -0.0107, -0.0164, -0.0164,
           -0.0077, -0.0155, -0.0143, -0.0083, -0.0137, -0.0142]],

         [[-0.0068, -0.0097, -0.0082, -0.0022, -0.0086, -0.0145, -0.0072,
           -0.0154, -0.0028, -0.0166, -0.0006, -0.0032, -0.0070, -0.0038,
           -0.0076, -0.0042, -0.0016, -0.0020, -0.0104, -0.0084],
          [-0.0163, -0.0011, -0.0035, -0.0078, -0.0050, -0.0164, -0.0151,
           -0.0049, -0.0094, -0.0016, -0.0101, -0.0099, -0.0093, -0.

In [115]:
actions.shape

torch.Size([5, 1, 3, 10])